# Factorization Machines

## Introduction

在现有互联网推荐系统中，存在许多特征，但这些特征的分布都是庞大而稀疏的，大量的研究和实际数据分析结果表明：某些特征之间的关联信息（相关度）对事件结果的的发生会产生很大的影响。从实际业务线的广告点击数据分析来看，也证实了这样的结论。于是Steffen Rendle提出了因子分解机（Factorization Machine），旨在解决大规模稀疏数据下的特征组合问题。

为何特征的分布都是庞大而稀疏的呢？我们从信息网站获取一个用户访问频道的日志，可以发现许多特征是类别特征，例如：

\begin{align}
{sports,news,finance,house,edu,......}
\end{align}

我们假设频道总数有10个，为${“auto”,“finance”,“ent”,“news”,“sports”,“mil”,“weather”,“house”,“edu”,“games”}$,则有训练数据如下：

user|channel
 -|-|
1|finance
2|ent
3|news
4|sports
...|...

特征ETL过程中，需要对categorical型特征进行one-hot编码（独热编码），即将categorical型特征转化为数值型特征。独热编码即 One-Hot 编码，又称一位有效编码，其方法是使用N位状态寄存器来对N个状态进行编码，每个状态都由他独立的寄存器位，并且在任意时候，其中只有一位有效。

> $自然状态码为：000,001,010,011,100,101$

> $独热编码为：000001,000010,000100,001000,010000,100000$

可以这样理解，对于每一个特征，如果它有m个可能值，那么经过独热编码后，就变成了m个二元特征（如成绩, 这个特征有好，中，差，变成one-hot就对应100, 010, 001）。并且，这些特征互斥，每次只有一个激活。因此，数据会变成稀疏的。

因此，对上述channel特征转化后的结果如下：

user|auto|finance|ent|news|sports|mil|weather|house|edu|games
-|-|-|-|-|-|-|-|-|-|-|
1|0|1|1|0|0|0|0|0|0|0
2|0|0|0|0|1|1|0|0|0|0
3|0|0|1|0|0|0|0|1|0|0
4|0|0|0|0|0|0|1|0|0|0
5|0|0|0|1|1|0|0|0|0|0
6|0|0|1|0|1|0|0|0|0|1
...|1|1|1|0|0|0|0|0|0|0

上面的例子中，一维categorical特征在经过one-hot编码后变成了10维数值型特征。真实应用场景中，未编码前特征总维度可能仅有数十维或者到数百维的categorical型特征，经过one-hot编码后，达到数千万、数亿甚至更高维度的数值特征在业内都是常有的。

此外也能发现，特征空间增长的维度取决于categorical型特征的取值个数,对应数量级会从$n$增长至$n^2$。

为了利用这些特征来提升learning performance，或许在学习过程中应考虑特征之间的关联信息，而表示特征之间的关联，最直接的方法的是构造组合特征。

## FM model

### Second order polynomial model

二阶多项式模型的表达式：

\begin{align}
\hat{y}(x) := \underbrace {w_0 + \sum_{i=1}^{n} w_i x_i }_{\text{线性回归}} + \underbrace {\sum_{i=1}^{n} \sum_{j=i+1}^{n} w_{ij} x_i x_j}_{\text{交叉项（组合特征）}}
\end{align}

其中，n 代表样本的特征数量，$x_i$是第 i 个特征的值,$w_0、w_i、w_{ij}$为模型参数。

由此公式可以看出，组合特征的数量为$\frac{n(n-1)}{2}$个，且任意两个交叉项参数$w_{ij}$相互独立。但由于样本数据很稀疏，$x_i x_j$为非零的项会非常的少，导致训练样本的不足，很容易导致参数 $w_{ij}$不准确，最终将严重影响模型的性能和稳定性。

因此我们需要用到矩阵分解，正如在一个rating矩阵可以分解为user矩阵和item矩阵，交叉项系数$w_{ij}$也可以组成一个对称矩阵$\mathbf{W}$，而其可以分解为：

\begin{align}
\mathbf{W} = \mathbf{V}^T \mathbf{V}, \mathbf{V}\in \mathbf{R}^{n\times k}
\end{align}

可以看出，$\mathbf{V}$中的第i列便是第i维的隐向量，即$w_{ij}=⟨v_i,v_j⟩$,这便是FM模型的核心思想。

### FM model

根据上述表达（只考虑二阶的情况），我们可以得到：

\begin{align}
\hat{y}(\mathbf{x}) := w_0 + \sum_{i=1}^{n} w_i x_i + \sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \mathbf{v}_i, \mathbf{v}_j \rangle x_i x_j
\end{align}

其中$\langle \rangle$表示两个向量的点积,即：

\begin{align}
\langle \mathbf{v}_i, \mathbf{v}_j \rangle := \sum_{f=1}^{k} v_{i,f} \cdot v_{j,f}
\end{align}

直观来讲，FM的复杂度为$O(kn^2)$，但是，通过下列等式，FM的二次项可以化简，其复杂度可以优化到$O(kn)$:

\begin{align}
\sum_{i=1}^n \sum_{j=i+1}^n \langle \mathbf{v}_i, \mathbf{v}_j \rangle x_i x_j = \frac{1}{2} \sum_{f=1}^k \left(\left( \sum_{i=1}^n v_{i, f} x_i \right)^2 - \sum_{i=1}^n v_{i, f}^2 x_i^2 \right)
\end{align}

证明如下:

>\begin{align} & \sum_{i=1}^{n} \sum_{j=i+1}^{n} {\langle \mathbf{v}_i, \mathbf{v}_j \rangle} x_i x_j \qquad\qquad\qquad\qquad\qquad\qquad(1)\\ = & \frac{1}{2} \sum_{i=1}^{n} \sum_{j=1}^{n} {\langle \mathbf{v}_i, \mathbf{v}_j \rangle} x_i x_j - \frac{1}{2} \sum_{i=1}^{n} {\langle \mathbf{v}_i, \mathbf{v}_i \rangle} x_i x_i \qquad\qquad\;\;(2)\\ = & \frac{1}{2} \left(\sum_{i=1}^{n} \sum_{j=1}^{n} \sum_{f=1}^{k} v_{i,f} v_{j,f} x_i x_j - \sum_{i=1}^{n} \sum_{f=1}^{k} v_{i,f} v_{i,f} x_i x_i \right) \qquad\,(3) \\ = & \frac{1}{2} \sum_{f=1}^{k} {\left \lgroup \left(\sum_{i=1}^{n} v_{i,f} x_i \right) \cdot \left(\sum_{j=1}^{n} v_{j,f} x_j \right) - \sum_{i=1}^{n} v_{i,f}^2 x_i^2 \right \rgroup} \quad\;\;\,(4) \\ = & \frac{1}{2} \sum_{f=1}^{k}  {\left \lgroup \left(\sum_{i=1}^{n} v_{i,f} x_i \right)^2 - \sum_{i=1}^{n} v_{i,f}^2 x_i^2\right \rgroup} \qquad\qquad\qquad\;\;(5) \end{align}

>其中第（1）步到第（2）步，这里用AA表示系数矩阵VV的上三角元素，BB表示对角线上的交叉项系数。由于系数矩阵VV是一个对称阵，所以下三角与上三角相等，有下式成立： 
\begin{align}
A = \frac{1}{2} (2A+B) - \frac{1}{2} B.  \quad \underline{ A=\sum_{i=1}^{n} \sum_{j=i+1}^{n} {\langle \mathbf{v}_i, \mathbf{v}_j \rangle} x_i x_j } ; \quad \underline{ B = \frac{1}{2} \sum_{i=1}^{n} {\langle \mathbf{v}_i, \mathbf{v}_i \rangle} x_i x_i }
\end{align}


采用随机梯度下降的参数梯度公式为：

\begin{align}
\frac{\partial}{\partial \theta} y(\mathbf{x}) = \left \{ \begin{array}{ll} 1,         & \text{if}\; \theta\; \text{is}\; w_0 \qquad \text{(常数项)} \\ x_i     & \text{if}\; \theta\; \text{is}\; w_i \;\qquad \text{(线性项)} \\ x_i \sum_{j=1}^{n} v_{j,f} x_j - v_{i,f} x_i^2, & \text{if}\; \theta\; \text{is}\; v_{i,f} \qquad \text{(交叉项)} \end{array} \right.
\end{align}